In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!pip install transformers

     |████████████████████████████████| 368kB 6.9MB/s 
     |████████████████████████████████| 1.0MB 46.1MB/s 
     |████████████████████████████████| 860kB 43.3MB/s 
     |████████████████████████████████| 675kB 48.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=d8cf9b41a2a61f53e3718cce79c94b34a155d60455f5abc78e20eaf2c8a2d40a
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
  Created wheel for regex: filename=regex-2019.12.9-cp36-cp36m-linux_x86_64.whl size=609170 sha256=467b0dd265dce41e3701720c2024077468debe3f6a762edae1179f108cb7ac77
  Stored in directory: /root/.cache/pip/wheels/0d/fb/b3/a89169557229468c49ca64f6839418f22461f6ee0a74f342b1
Successfully built sacremoses regex


In [3]:
!pip install -U sentence-transformers

     |████████████████████████████████| 51kB 3.1MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.2.4.1-cp36-none-any.whl size=61094 sha256=327feef77ce295f15a2e83ae18a1374bc16fa02836c876b40b6e9ff74ea7e5b9
  Stored in directory: /root/.cache/pip/wheels/12/a5/1c/03b7d87e027121fe1e23048007594e73f39a23e833658529c7
Successfully built sentence-transformers


In [4]:
"""
This examples trains BERT for the STSbenchmark from scratch. It generates sentence embeddings
that can be compared using cosine-similarity to measure the similarity.
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, models
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import STSDataReader
import logging
from datetime import datetime


#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

In [6]:
# Read the dataset
train_batch_size = 16
num_epochs = 4
model_save_path = 'output/training_n2c2_sts_bert-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# parameters for STSDataReader -> dataset_folder, s1_col_idx=5, s2_col_idx=6, score_col_idx=4, delimiter="\t", quoting=csv.QUOTE_NONE, normalize_scores=True, min_score=0, max_score=5
sts_reader = STSDataReader('', s1_col_idx=0, s2_col_idx=1, score_col_idx=2, normalize_scores=True)

# Use BERT for mapping tokens to embeddings
word_embedding_model = models.BERT('bert-base-uncased')

# Apply mean pooling to get one fixed sized sentence vector
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=True,
                               pooling_mode_max_tokens=True)

#model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

dense1 = models.Dense(in_features=3*word_embedding_model.get_word_embedding_dimension(), out_features=256)
dense2 = models.Dense(in_features=256, out_features=64)
#dense3 = models.Dense(in_features=64, out_features=8)

model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense1, dense2])

2019-12-11 19:49:26 - https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to /tmp/tmpvw87dl2v


100%|██████████| 313/313 [00:00<00:00, 139394.47B/s]


2019-12-11 19:49:26 - copying /tmp/tmpvw87dl2v to cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
2019-12-11 19:49:26 - creating metadata file for /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
2019-12-11 19:49:26 - removing temp file /tmp/tmpvw87dl2v
2019-12-11 19:49:26 - loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /root/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
2019-12-11 19:49:26 - Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0

100%|██████████| 440473133/440473133 [00:16<00:00, 26340603.89B/s]


2019-12-11 19:49:43 - copying /tmp/tmpnx3hazhg to cache at /root/.cache/torch/transformers/aa1ef1aede4482d0dbcd4d52baad8ae300e60902e88fcb0bebdec09afd232066.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157
2019-12-11 19:49:45 - creating metadata file for /root/.cache/torch/transformers/aa1ef1aede4482d0dbcd4d52baad8ae300e60902e88fcb0bebdec09afd232066.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157
2019-12-11 19:49:45 - removing temp file /tmp/tmpnx3hazhg
2019-12-11 19:49:45 - loading weights file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-pytorch_model.bin from cache at /root/.cache/torch/transformers/aa1ef1aede4482d0dbcd4d52baad8ae300e60902e88fcb0bebdec09afd232066.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157
2019-12-11 19:49:48 - https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache or force_download set to True, downloading to /tmp/tmp3lp2lm0x


100%|██████████| 231508/231508 [00:00<00:00, 928045.50B/s]


2019-12-11 19:49:48 - copying /tmp/tmp3lp2lm0x to cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2019-12-11 19:49:48 - creating metadata file for /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2019-12-11 19:49:48 - removing temp file /tmp/tmp3lp2lm0x
2019-12-11 19:49:48 - loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /root/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
2019-12-11 19:49:48 - Use pytorch device: cuda


In [8]:
#sts_reader = STSDataReader('', s1_col_idx=0, s2_col_idx=1, score_col_idx=2, normalize_scores=True)

my_drive = '/content/gdrive/My Drive/'

# Convert the dataset to a DataLoader ready for training
logging.info("Read STSbenchmark train dataset")
train_data = SentencesDataset(sts_reader.get_examples(my_drive+'clinicalSTS2019.train_train60.csv'), model)
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)


logging.info("Read STSbenchmark dev dataset")
dev_data = SentencesDataset(examples=sts_reader.get_examples(my_drive+'clinicalSTS2019.train_validate20.csv'), model=model)
dev_dataloader = DataLoader(dev_data, shuffle=False, batch_size=train_batch_size)
evaluator = EmbeddingSimilarityEvaluator(dev_dataloader)


2019-12-11 20:00:09 - Read STSbenchmark train dataset


Convert dataset: 100%|██████████| 992/992 [00:01<00:00, 978.48it/s] 


2019-12-11 20:00:10 - Num sentences: 992
2019-12-11 20:00:10 - Sentences 0 longer than max_seqence_length: 0
2019-12-11 20:00:10 - Sentences 1 longer than max_seqence_length: 0
2019-12-11 20:00:10 - Read STSbenchmark dev dataset


Convert dataset: 100%|██████████| 331/331 [00:00<00:00, 1021.61it/s]

2019-12-11 20:00:11 - Num sentences: 331
2019-12-11 20:00:11 - Sentences 0 longer than max_seqence_length: 0
2019-12-11 20:00:11 - Sentences 1 longer than max_seqence_length: 0


In [9]:
# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_data)*num_epochs/train_batch_size*0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))


2019-12-11 20:00:11 - Warmup-steps: 25


In [10]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

Convert Evaluating:   0%|          | 0/21 [00:00<?, ?it/s]

2019-12-11 20:00:31 - Evaluation the model on  dataset after epoch 0:



Epoch:   0%|          | 0/4 [00:22<?, ?it/s]

2019-12-11 20:00:33 - Cosine-Similarity :	Pearson: 0.7004	Spearman: 0.7279
2019-12-11 20:00:33 - Manhattan-Distance:	Pearson: 0.7013	Spearman: 0.7068
2019-12-11 20:00:33 - Euclidean-Distance:	Pearson: 0.7065	Spearman: 0.7098
2019-12-11 20:00:33 - Dot-Product-Similarity:	Pearson: 0.5159	Spearman: 0.5702
2019-12-11 20:00:33 - Save model to output/training_n2c2_sts_bert-2019-12-11_19-49-25
2019-12-11 20:00:33 - Configuration saved in output/training_n2c2_sts_bert-2019-12-11_19-49-25/0_BERT/config.json


Epoch:  25%|██▌       | 1/4 [00:23<01:09, 23.01s/it]

Iteration:   0%|          | 0/62 [00:00<?, ?it/s]

...

2019-12-11 20:01:18 - Cosine-Similarity :	Pearson: 0.6836	Spearman: 0.6832
2019-12-11 20:01:18 - Manhattan-Distance:	Pearson: 0.6172	Spearman: 0.6054
2019-12-11 20:01:18 - Euclidean-Distance:	Pearson: 0.6174	Spearman: 0.6021
2019-12-11 20:01:18 - Dot-Product-Similarity:	Pearson: 0.5488	Spearman: 0.5987
2019-12-11 20:01:18 - Restart data_iterator



Convert Evaluating:   0%|          | 0/21 [00:00<?, ?it/s]

2019-12-11 20:01:39 - Evaluation the model on  dataset after epoch 3:



Epoch: 100%|██████████| 4/4 [01:29<00:00, 22.46s/it]

2019-12-11 20:01:41 - Cosine-Similarity :	Pearson: 0.7152	Spearman: 0.7195
2019-12-11 20:01:41 - Manhattan-Distance:	Pearson: 0.6525	Spearman: 0.6459
2019-12-11 20:01:41 - Euclidean-Distance:	Pearson: 0.6573	Spearman: 0.6448
2019-12-11 20:01:41 - Dot-Product-Similarity:	Pearson: 0.5986	Spearman: 0.6413


In [13]:
##############################################################################
#
# Load the stored model and evaluate its performance on STS benchmark dataset
#
##############################################################################

model = SentenceTransformer(model_save_path)
test_data = SentencesDataset(examples=sts_reader.get_examples(my_drive+"clinicalSTS2019.train_test20.csv"), model=model)
test_dataloader = DataLoader(test_data, shuffle=False, batch_size=train_batch_size)
evaluator = EmbeddingSimilarityEvaluator(test_dataloader)
model.evaluate(evaluator)

2019-12-11 20:09:36 - Load pretrained SentenceTransformer: output/training_n2c2_sts_bert-2019-12-11_19-49-25
2019-12-11 20:09:36 - Load SentenceTransformer from folder: output/training_n2c2_sts_bert-2019-12-11_19-49-25
2019-12-11 20:09:36 - loading configuration file output/training_n2c2_sts_bert-2019-12-11_19-49-25/0_BERT/config.json
2019-12-11 20:09:36 - Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

2019-12-11 20:09:36 - loading weights file output/training_n2

Convert Evaluating:  10%|▉         | 2/21 [00:00<00:01, 10.68it/s]

2019-12-11 20:09:39 - Num sentences: 331
2019-12-11 20:09:39 - Sentences 0 longer than max_seqence_length: 0
2019-12-11 20:09:39 - Sentences 1 longer than max_seqence_length: 0
2019-12-11 20:09:39 - Evaluation the model on  dataset:


Convert Evaluating: 100%|██████████| 21/21 [00:01<00:00, 12.11it/s]

2019-12-11 20:09:41 - Cosine-Similarity :	Pearson: 0.6864	Spearman: 0.6851
2019-12-11 20:09:41 - Manhattan-Distance:	Pearson: 0.6813	Spearman: 0.6606
2019-12-11 20:09:41 - Euclidean-Distance:	Pearson: 0.6854	Spearman: 0.6662
2019-12-11 20:09:41 - Dot-Product-Similarity:	Pearson: 0.5486	Spearman: 0.5511


0.6851091758294848

In [0]:
!zip -r output.zip output

In [0]:
!cp output.zip '/content/gdrive/My Drive/bert_finetuned_sts_biomedical_dense.zip'